<h1 style="font-size:30px;">Pipeline Configuration in TFOD </h1>

Every model present in **Model Zoo** of **TensorFlow Object Detection API** comes with a configuration file called `pipeline.config` file. It let's us modify different parameters of the model like number of classes, batch size etc. We will also need to edit this file to add paths to our training dataset and other required files.

<img src = "https://opencv.org/wp-content/uploads/2022/07/c4_06_pipeline_config.png" align='center' width=400>

# Table of Contents

* [1 Install the TFOD API](#1-Install-the-TFOD-API)
* [2 Download the Model](#2-Download-the-Model)
* [3 Important Config Parameters](#3-Important-Config-Parameters)
* [4 Label Map](#4-Label-Map)
* [5 Additional Params](#5-Additional-Params)
* [6 Save the edited Config File](#6-Save-the-edited-Config-File)
* [7 Conclusion](#7-Conclusion)

## 1 Install the TFOD API

**Clone the GitHub Repo**

Here we will clone the TFOD API repo.

In [ ]:
import os

if not 'models' in os.listdir():
  !git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 3451, done.
remote: Counting objects: 100% (3451/3451), done.
remote: Compressing objects: 100% (2891/2891), done.
remote: Total 3451 (delta 891), reused 1416 (delta 503), pack-reused 0
Receiving objects: 100% (3451/3451), 46.85 MiB | 32.20 MiB/s, done.
Resolving deltas: 100% (891/891), done.


In [ ]:
# The `%%bash` magic command inside a notebook lets you run a cell run like a shell interface
# Note: the `bash` command works only on Colab.
%%bash 

# Change the directory to models/research
cd models/research/

# Compile the API's Protobuf files
protoc object_detection/protos/*.proto --python_out=.

# Copy the required Setup file
cp object_detection/packages/tf2/setup.py .

# Install the API using the setup.py file
python -m pip install .

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/models/research
  Created wheel for object-detection: filename=object_detection-0.1-py3-none-any.whl size=1694955 sha256=55f5f60c93c977e676f81ac53806df217b61f76e9ebf7a01205b8d60d356cb93
  Stored in directory: /tmp/pip-ephem-wheel-cache-daq6g6rf/wheels/fa/a4/d2/e9a5057e414fd46c8e543d2706cd836d64e1fcd9eccceb2329
  Created wheel for py-cpuinfo: filename=py_cpuinfo-8.0.0-py3-none-any.whl size=22257 sha256=f334fc20ab536d37b59f514595527d81b9b1c77417cd601f35358f3e8cb34b60
  Stored in directory: /root/.cache/pip/wheels/d2/f1/1f/041add21dc9c4220157f1bd2bd6afe1f1a49524c3396b94401
  Created wheel for dill: filename=dill-0.3.1.1-py3-none-any.whl size=78544 sha256=76ebc4174b5bb2a9d0c75345499e7f89ed9066ebd4f2bbffb8732a79a0d970d6
  Stored in directory: /root/.cache/pip/wheels/a4/61/fd/c57e374e580aa78a45ed78d5859b3a44436af17e22ca53284f
  Created wheel for avro-python3: filename=avro_p

  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gym 0.17.3 requires cloudpickle<1.7.0,>=1.2.0, but you have cloudpickle 2.1.0 which is incompatible.


In this notebook we will go over different paramaters that can be modified in this file. We will also provide a description of each parameter and also share code to modify those parameters. 

We should note that the `pipeline.config` file enables us to modify almost all the parameters in the file; but we shall restrict ourselves to only those parameters that would be required for fine tuning a pretrained model. 

Now you can open up the `pipeline.config` file in a text editor and change each value, but it is always not a wise solution to update the `config` file manually; however, **TFOD API** provides us with a convinient module called `config_util`. This convenient module allows us to pragramitically change all the required params.

We will start by importing the `config_util` module.

In [ ]:
import requests
import tarfile
import os

from object_detection.utils import config_util

## 2 Download the Model

First, let us download the Model from the official [TensorFlow Model Zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md) repo. You can use any model you like, we will be using a pre-trained **`RetinaNet_101`** model, which will be further used for fine-tuning.
 
This download model function downloads any model from the zoo.

In [ ]:
# Function to Download any model from Model Zoo from their URL
def download_model(model_name, url):

    file = requests.get(url)
    open(model_name+'.tar.gz', 'wb').write(file.content)

    # Extract the Model
    tar = tarfile.open(model_name + '.tar.gz')
    tar.extractall(model_name)
    tar.close

Download the **`RetinaNet_101`** model by calling the function above.

In [ ]:
# Define URL and name of the Model
model_url = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet101_v1_fpn_640x640_coco17_tpu-8.tar.gz'
# Directory where the model and its configuration should be downloaded.
model_directory = 'RetinaNet_101'

model_name = model_url.split('/')[-1].split('.')[0] # ssd_resnet101_v1_fpn_640x640_coco17_tpu-8

# Download RetinaNet 101 model from the Model Zoo
download_model(model_directory, model_url)

After that you'll need to load the `pipeline.config` file with it.

In [ ]:
# Define the path where pipeline config is present.
base_config_path = os.path.join(model_directory, model_name, 'pipeline.config')

# Read the config file in the form a dictionary
configs = config_util.get_configs_from_pipeline_file(base_config_path)

## 3 Important Config Parameters

In this notebook, we have used `ssd_resnet101_v1_fpn_640x640_coco17_tpu-8` as our baseline model. However, you can have a look at the config files of other models present in the model zoo of TFOD API.

Basically, the config files of almost all the object detection models in TFOD share the same file structure containing the following keys:

- **`model:`** Contains information about the model such as model architecture, loss functions used, post-processing info like NMS, etc


- **`train_config:`** Contains information about the train data such as batch size, number of steps to train for, optimizer used, data augmentations used, etc


- **`train_input_reader:`** Contains paths to label map and tf record for the training data


- **`eval_config:`** Contains information about the evaluation metric to be used


- **`eval_input_reader:`** Contains paths to label map and tf record for the validation data


For a clearer understanding, let us have a quick look about the model information

In [ ]:
print(configs['model'])

ssd {
  num_classes: 90
  image_resizer {
    fixed_shape_resizer {
      height: 640
      width: 640
    }
  }
  feature_extractor {
    type: "ssd_resnet101_v1_fpn_keras"
    depth_multiplier: 1.0
    min_depth: 16
    conv_hyperparams {
      regularizer {
        l2_regularizer {
          weight: 0.00039999998989515007
        }
      }
      initializer {
        truncated_normal_initializer {
          mean: 0.0
          stddev: 0.029999999329447746
        }
      }
      activation: RELU_6
      batch_norm {
        decay: 0.996999979019165
        scale: true
        epsilon: 0.0010000000474974513
      }
    }
    override_base_feature_extractor_hyperparams: true
    fpn {
      min_level: 3
      max_level: 7
    }
  }
  box_coder {
    faster_rcnn_box_coder {
      y_scale: 10.0
      x_scale: 10.0
      height_scale: 5.0
      width_scale: 5.0
    }
  }
  matcher {
    argmax_matcher {
      matched_threshold: 0.5
      unmatched_threshold: 0.5
      ignore_thresholds: 

Now let’s take a look at some basic and the most important configurations required for fine-tuning.

###  3.1 **`num_classes`**

This parameter is used to set the number of classes according to the dataset that is being used.

In [ ]:
def update_num_classes(model_config, num_classes):
    """
    Args: 
    model_config: model_pb2.DetectionModel object.
    num_classes: `int` indicating the number of classes to set.
    """
    
    meta_architecture = model_config.WhichOneof("model")

    if meta_architecture == "faster_rcnn":
        model_config.faster_rcnn.num_classes = num_classes
    elif meta_architecture == "ssd":
        model_config.ssd.num_classes = num_classes
    elif meta_architecture == "center_net":
        model_config.center_net.num_classes = num_classes

    return

In [ ]:
# Total classes in your dataset 
num_classes = 4

update_num_classes(configs['model'], num_classes)

### 3.2  **`fine_tune_checkpoint`**

We set this parameter to the path of our pre-trained model’s checkpoint so that we can use what it has learned to solve our problem and get better results.

In [ ]:
def update_fine_tune_checkpoint_path(train_config, checkpoint_path):
    """
    Args: 
    train_config: train_pb2.TrainConfig object.
    checkpoint_path: path to pre-trained model’s checkpoint.
    """
    
    train_config.fine_tune_checkpoint = checkpoint_path

    return

In [ ]:
fine_tune_checkpoint_path = os.path.join(model_directory, model_name, 'checkpoint', 'ckpt-0')

update_fine_tune_checkpoint_path(configs['train_config'], fine_tune_checkpoint_path)

3.3 **`num_steps`**

`num_steps` are the number of total training steps for the model. In each step the model processes data samples that is equivalent to the batch size. So basically, `num_steps` is the total number of training iterations.

**Note:** The `num_steps` is not equivalent to the number of epochs for training.

In [ ]:
num_steps = 10000

config_util._update_train_steps(configs, num_steps)

### 3.4 **`batch_size`**

Ths is the number of training samples utilized in one training step.

In [ ]:
batch_size = 8

config_util._update_batch_size(configs, batch_size)

### 3.5 **`train_input_reader`**

The API has different input readers for taking training and testing data. The `train_input_reader` requires the path to our TF Records File which contains the data we need to use for training. So here we pass `train_input_config` to update the path to our training data

In [ ]:
train_record_path = 'train_data.tfrecord'

config_util.update_input_reader_config(configs,
                                       'train_input_config',
                                       'tf_record_input_reader',
                                       'input_path',
                                        train_record_path,                                  
                                       )

Note that while updating the path to our tfrecord file we follow a hierarchy that is also followed in the `pipeline.config` file. And since we had to change the training data we use `train_input_config` in the second parameter. We will set a different name when updating the TFrecords path for validation data.


### 3.6 **`eval_input_reader`**


Here is another input reader. This one takes in validation data. Note that here we pass `eval_input_config` as one of the argument. Other than that it follows the same hierarchy as `train_input_reader`.

In [ ]:
val_record_path = 'val_data.tfrecord'

config_util.update_input_reader_config(configs,
                                       'eval_input_config',
                                       'tf_record_input_reader',
                                       'input_path',
                                       val_record_path,
                                       )

### 3.7 **`shuffle`**


Set this param to `true` to randomize the dataset for better evaluation. We use this parameter to randomize the order of the data on which evaluation is performed at every step so we can better train and judge the model on its accuracy. 


In [ ]:
shuffle_flag = False

config_util.update_input_reader_config(configs, 
                                       'eval_input_config',
                                       None,
                                       'shuffle',
                                       shuffle_flag
                                       )

### 3.8 **`fine_tune_checkpoint_type`**

We set this param to let our model know what kind of weights we want to use from the pre-trained model and what type of task we are going to train it for. In our case it's a detection task.

You can know more about the other supported checkpoint types from [here](https://github.com/tensorflow/models/blob/master/research/object_detection/protos/train.proto#L43).



In [ ]:
def update_fine_tune_checkpoint_type(train_config, fine_tune_checkpoint_type):
    """
    Args: 
    train_config: train_pb2.TrainConfig object.
    fine_tune_checkpoint_type: determines the type of weights that are restored from 
                               from the pre-trained fine_tune_checkpoint.
                               Can be either of: "classification", "detection" or "full".
    """
    
    train_config.fine_tune_checkpoint_type = fine_tune_checkpoint_type
    
    return

In [ ]:
fine_tune_checkpoint_type = 'detection' 

update_fine_tune_checkpoint_type(configs['train_config'], fine_tune_checkpoint_type)

## 4 Label Map

TensorFlow Object Detection API requires a label map which maps each of the labels to integer values. This labeled map is used at both times, during training and evaluation.

Below we show an example label map (e.g `label_map.pbtxt`), assuming that our dataset contains 5 labels, `Ambulance`, `Bus`, `Car`, `Motorcycle` and `Truck`:

In [ ]:
# Create Label Map of the Dataset
pbtxt = '''
item {
    name: 'car',
    id: 1,
}

item {
    name: 'dog',
    id: 2,
}

item {
    name: 'person',
    id: 3,
}

item {
    name: 'tvmonitor',
    id: 4,
}
'''

# Save this labelmap to disk
with open("labelmap.pbtxt", "w") as text_file:
    text_file.write(pbtxt)

Label map files have the extension `.pbtxt`.

**Note:** We start the label map from `1` because `0` is reserved for the background. Anything that is not part of the orginal classes is regarded as background and stored as `0`.

Now we have to give this labelmap to our model's configuration file so our model is aware of the classes we are using and what their class ids are.


In [ ]:
labelmap_path = 'labelmap.pbtxt'

config_util._update_label_map_path(configs, labelmap_path)

## 5 Additional Params

Beside's the above parameters there are lot's of other Additional `pipeline.config` parameters that you need not modify with `config_util` because they already have good default values but it's worth discussing a few of them. 

### 5.1 `data_augmentation_options`

Our config file has the following augmentation steps:

```
data_augmentation_options {
    random_horizontal_flip {
    }
  }
  
data_augmentation_options {
    random_crop_image {
      min_object_covered: 0.0
      min_aspect_ratio: 0.75
      max_aspect_ratio: 3.0
      min_area: 0.75
      max_area: 1.0
      overlap_thresh: 0.0
    }
  }
```

You can try out a variety of augmentations provided in the API. To see the list of augmentation techniques supported by TFOD API check [here](https://github.com/tensorflow/models/blob/238922e98dd0e8254b5c0921b241a1f5a151782f/research/object_detection/protos/preprocessor.proto)

Let's illustrate the explainations through some example. 


In [ ]:
from object_detection.protos import preprocessor_pb2 as preprocess

Suppose we want to add `random_adjust_contrast` as our augmentation step with the default values only. The following code cell can be used to perform this task.

In [ ]:
adj_contrast = preprocess.RandomAdjustContrast()

preprocess_step = preprocess.PreprocessingStep()
# Copy all the default values
preprocess_step.random_adjust_contrast.CopyFrom(adj_contrast)

Next, we append this preprocessing step to our `data_augmentation_options` container.

In [ ]:
configs['train_config'].data_augmentation_options.append(preprocess_step)

Say, we want to update the parameters for a particular data augmentation manually. We can do this task, by executing the code cell below!

In [ ]:
# Update the values manually
preprocess_step.random_adjust_saturation.min_delta = 0.5
preprocess_step.random_adjust_saturation.max_delta = 1.0

As discussed above, we will append it to our `data_augmentation_options` container.

In [ ]:
configs['train_config'].data_augmentation_options.append(preprocess_step)

Finally, let's take a look at all the data augmentations that have been defined.

In [ ]:
configs['train_config'].data_augmentation_options

[random_horizontal_flip {
}
, random_crop_image {
  min_object_covered: 0.0
  min_aspect_ratio: 0.75
  max_aspect_ratio: 3.0
  min_area: 0.75
  max_area: 1.0
  overlap_thresh: 0.0
}
, random_adjust_contrast {
}
, random_adjust_saturation {
  min_delta: 0.5
  max_delta: 1.0
}
]

### 5.2  **`image_resizer`**

Every model’s config file will come with a set image size which all the images will be resized to during training.

Syntax from config file:

```
image_resizer {
      fixed_shape_resizer {
        height: 640
        width: 640
      }
    }
```
Generally the config files comes with 2 types of `images_resizer`:

- `fixed_shape_resizer` : reshapes the images to a fixed size as specified in the `height` and `width` parameters


- `keep_aspect_ratio_resizer`: defined by `min_dimension` and a `max_dimesion` for all images. These are scalar values. For e.g. if the `min_dimension` is a scalar value of `500` then during the training, the model will make sure that to resize the smaller dimension of an image while keeping the aspect ratio.

According to the documentation provided in the [resize_to_range](https://github.com/tensorflow/models/blob/238922e98dd0e8254b5c0921b241a1f5a151782f/research/object_detection/core/preprocessor.py#L2895) function for resizing, it states:

```
The output size can be described by two cases:
  1. If the image can be rescaled so its minimum dimension is equal to the
     provided value without the other dimension exceeding max_dimension,
     then do so.
  2. Otherwise, resize so the largest dimension is equal to max_dimension.
```

Additionally we have a parameter `pad_to_max_dimension` for configurations with `keep_aspect_ratio_resizer` that resizes the image and pad it with zeros so the resulting image is of the spatial size `[max_dimension, max_dimension]`

In [ ]:
# You can use this to check the input size details.
config_util.get_image_resizer_config(configs['model'])

fixed_shape_resizer {
  height: 640
  width: 640
}

Now, let's update the image dimensions for the model configuration. We will subsequently call the `update_image_resizer`  function for this task.

In [ ]:
def update_image_resizer(model_config, min_dim = None, max_dim = None, height = None, width = None):
    """
    Args: 
    model_config: model_pb2.DetectionModel object.
    min_dim: `int` ---> Desired size of the smaller image dimension in pixels.
    max_dim: `int` ---> Desired size of the smaller image dimension in pixels.
    height:  `int` ---> Desired height of image in pixels.
    width:   `int` ---> Desired width of image in pixels.
    """
    
    meta_architecture = model_config.WhichOneof("model")
    print('Model Architecture: ', meta_architecture)
    model = getattr(model_config,meta_architecture)
    if model.image_resizer.HasField('fixed_shape_resizer'):
        model.image_resizer.fixed_shape_resizer.height = height
        model.image_resizer.fixed_shape_resizer.width = width
            
    elif model.image_resizer.HasField('keep_aspect_ratio_resizer'):
        model.image_resizer.keep_aspect_ratio_resizer.min_dimension = min_dim 
        model.image_resizer.keep_aspect_ratio_resizer.max_dimension = max_dim
    return

In [ ]:
update_image_resizer(configs['model'], height = 780, width = 900 )
config_util.get_image_resizer_config(configs['model'])

Model Architecture:  ssd


fixed_shape_resizer {
  height: 780
  width: 900
}

### 5.3 Classification Loss and Localization Loss

In Object Detection, we use two heads at the end of the network. One is for prediction of bounding boxes and one is for classification. Learning both of these requires the network to have two different losses for each of the heads. So we have a classification loss and a localization loss. 

There is also a weight we assign to both losses and by default in the TFOD API it is set to 1.0 for both of them because we want to give them equal weightage(importance) during training; such that the total loss is calculated in terms of: `localization_weight * localization_loss + classification_weight * classification loss` (along with `regularization_loss` if enabled)


```
loss {
      localization_loss {
        weighted_smooth_l1 {
        }
      }
      classification_loss {
        weighted_sigmoid_focal {
          gamma: 2.0
          alpha: 0.25
        }
      }
      classification_weight: 1.0
      localization_weight: 1.0
    }
```

You can take a look at the other losses that are available in the [losses.proto](https://github.com/tensorflow/models/blob/master/research/object_detection/protos/losses.proto). However, it is not recommended to change the losses for the already pre-trained models.

Now, let's update the localization and classification weights for our model configuration. We will use the `update_loc_cls_wts` function for this task.

In [ ]:
def update_loc_cls_wts(model_config, loc_weight=1.0, cls_weight=1.0):
    """
    Args: 
    model_config: model_pb2.DetectionModel object.
    loc_weight: `float` ---> Localization loss weight.
    cls_weight: `float` ---> Classification loss weight.
    """
    
    meta_architecture = model_config.WhichOneof("model")

    if meta_architecture == "faster_rcnn":
        model_config.faster_rcnn.second_stage_localization_loss_weight = loc_weight
        model_config.faster_rcnn.second_stage_classification_loss_weight = cls_weight
        
        
    elif meta_architecture == "ssd":
        model_config.ssd.loss.localization_weight = loc_weight
        model_config.ssd.loss.classification_weight = cls_weight

    return

In [ ]:
update_loc_cls_wts(configs['model'], loc_weight=15.0, cls_weight=1.5)

Let's take a look at our updates for the localization and classification weights.

In [ ]:
configs['model'].ssd.loss

localization_loss {
  weighted_smooth_l1 {
  }
}
classification_loss {
  weighted_sigmoid_focal {
    gamma: 2.0
    alpha: 0.25
  }
}
classification_weight: 1.5
localization_weight: 15.0

### 5.4 Learning Rate

The learning rate is a hyperparameter that controls how much to tune the model in response to the prediction error each time the model weights are modified on a single training step.

This is a really important parameter that decides how fast your model learns, failing to set this parameter properly can give you a lot of trouble.

**TFOD API** employs some complex learning schemes to select and change the learning rate througout the training. 


This includes a warm learning rate, warm up steps, the actual learning rate, a learning_rate_decay etc. 

```
optimizer {
    momentum_optimizer {
      learning_rate {
        cosine_decay_learning_rate {
          learning_rate_base: 0.04
          total_steps: 25000
          warmup_learning_rate: 0.013333
          warmup_steps: 2000
        }
      }
      momentum_optimizer_value: 0.9
    }
    use_moving_average: false
  }
```

Lets go over the additional parameters within learning rate:

* **Base Learning Rate**
    This value is the rate at which the model learns normally for the whole training process if a **warm up learning rate** is not set.
  
* **Warm Up Learning Rate**
    Whenever we start the model for training, it is often a good practice to initially let the model learn slowly so it can learn from the early examples correctly. After the model has learned for a set number of steps. We change the learning rate to a value which lets our model learn and reach convergance faster.

* **Total Steps**
    This parameter is the same as the the same as `num_steps` parameter we studied above. 

* **Warmup Steps**
    These are the number of steps during which the learning rate starts from the `warmup_learning_rate` and goes till the `learning_rate_base`.

In [ ]:
# Initial LR
initial_lr = 0.0003

# Update Initial LR
config_util._update_initial_learning_rate(configs, initial_lr)

### 5.5 Feature Extractor

A pre-trained model is a network that was previously trained on the MS COCO 2017. You either use the pretrained model as is or use transfer learning to customize this model for a given task. 

The feature extractor is the name of the pretrained model that is being used for fine_tuning. This is also referred as the back bone model or the base model.

It useful to check what backbone is being used for your detection task.

For instance, if we are using a pre-trained **RetinaNet101**, the backbone is a `ssd_resnet101_v1_fpn_keras`. 

```
feature_extractor {
      type: "ssd_resnet101_v1_fpn_keras"
      depth_multiplier: 1.0
      min_depth: 16
      conv_hyperparams {
        regularizer {
          l2_regularizer {
            weight: 0.0004
          }
        }
        initializer {
          truncated_normal_initializer {
            mean: 0.0
            stddev: 0.03
          }
        }
        activation: RELU_6
        batch_norm {
          decay: 0.997
          scale: true
          epsilon: 0.001
        }
      }
      override_base_feature_extractor_hyperparams: true
      fpn {
        min_level: 3
        max_level: 7
      }
    }
```

### 5.6 **`iou_threshold`**

In **Non-max Suppression**, we need `iou_threshold` to suppress less confident predictions in the case of less confident bounding box overlaps sufficiently with a more confident bounding box prediction. 

```
post_processing {
      batch_non_max_suppression {
        score_threshold: 1e-08
        iou_threshold: 0.6
        max_detections_per_class: 100
        max_total_detections: 100
        use_static_shapes: false
      }
      score_converter: SIGMOID
    }
```

In [ ]:
def update_iou_threshold(model_config, iou_thresh=0.5):
    """
    Args: 
    model_config: model_pb2.DetectionModel object.
    iou_thresh: `float` ---> IoU threshold required during Non-Maximum suppression.
    """
    
    meta_architecture = model_config.WhichOneof("model")

    if meta_architecture == "faster_rcnn":
        model_config.faster_rcnn.second_stage_post_processing.batch_non_max_suppression.iou_threshold = iou_thresh
        
        
    elif meta_architecture == "ssd":
        model_config.ssd.post_processing.batch_non_max_suppression.iou_threshold = iou_thresh        
  

    return

In [ ]:
update_iou_threshold(configs['model'], iou_thresh=0.4)

### 5.7 **`max_detections_per_class`**

This parameter allows you to set the maximum number of detections of a single class on an image. 

**e.g:** maximum 100 cats should be detected on each image.

In [ ]:
def update_max_detections_per_class(model_config, max_dets_per_class=100):
    """
    Args: 
    model_config: model_pb2.DetectionModel object.
    max_dets_per_class: `int` ---> Maximum number of detections to retain per class.
    """
    
    meta_architecture = model_config.WhichOneof("model")

    if meta_architecture == "faster_rcnn":
        model_config.faster_rcnn.second_stage_post_processing.batch_non_max_suppression.max_detections_per_class = max_dets_per_class
        
        
    elif meta_architecture == "ssd":
        model_config.ssd.post_processing.batch_non_max_suppression.max_detections_per_class = max_dets_per_class    
  

    return

In [ ]:
update_max_detections_per_class(configs['model'], max_dets_per_class=50)

### 5.8 **`max_total_detections`**

This parameter sets the maximum number of total detections which is the sum of all detections of all classes combined. 

In [ ]:
def update_max_total_detections(model_config, max_dets=100):
    """
    Args: 
    model_config: model_pb2.DetectionModel object.
    max_dets: `int` ---> Maximum number of detections to retain across all classes.
    """
    
    meta_architecture = model_config.WhichOneof("model")

    if meta_architecture == "faster_rcnn":
        model_config.faster_rcnn.second_stage_post_processing.batch_non_max_suppression.max_total_detections = max_dets
        
        
    elif meta_architecture == "ssd":
        model_config.ssd.post_processing.batch_non_max_suppression.max_total_detections = max_dets
  

    return

In [ ]:
update_max_total_detections(configs['model'], max_dets=50)

## 6 Save the edited Config File

After you're done making changing then it's worth mentioning that all the changes are made in memory and the actual file and you'll need to serialize it in a file. So first we have to create a config file containing those changes using `config_util.create`

And then we'll serialize this and with `config_util.save`, this will generate a new `pipeline.config` file on disk. 

In [ ]:
# Create a pipeline file instance from the edited configuration instance
configs_file = config_util.create_pipeline_proto_from_configs(configs)

# Save the pipeline into a directory
config_util.save_pipeline_config(configs_file, './')

## 7 Conclusion

In this notebook, we have learned the following with respect to the configuration parameters:

1. Modify the parameters with respect to the dataset for which we are training our model on such as: `num_classes`, `batch_size`, labelmap, etc.


2. Try out a variety of pretrained model checkpoints available from [TensorFlow 2 Model Zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md).


3. Explore additional parameters such as dat augmentations, `image_resizer`, etc.


4. Learn about model hyperparameters such as loss functions, learning rate optimizers, etc.